In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler,LabelEncoder
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [3]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [4]:
label_encoder=LabelEncoder()

In [5]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [6]:
dataframe=df

In [7]:
dataframe

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,1
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,1
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,1
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,1
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,1
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,1
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,1
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,1
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,1


In [9]:
features = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
labels = df['Class Name']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42,stratify=labels)


In [19]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [30]:
y_pred = rf_model.predict(X_test)


In [32]:
y_pred.shape

(10330,)

In [34]:
(y_pred==0).sum()

3126

In [35]:
(y_test==0).sum()

3196

In [21]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[3105   91]
 [  21 7113]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3196
           1       0.99      1.00      0.99      7134

    accuracy                           0.99     10330
   macro avg       0.99      0.98      0.99     10330
weighted avg       0.99      0.99      0.99     10330



In [23]:
import joblib

In [29]:
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))
joblib.dump(rf_model, 'random_forest_model.pkl')
num_params = sum(tree.tree_.node_count for tree in rf_model.estimators_)
print(f'Toplam parametre sayısı: {num_params}')

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3196
           1       0.99      1.00      0.99      7134

    accuracy                           0.99     10330
   macro avg       0.99      0.98      0.99     10330
weighted avg       0.99      0.99      0.99     10330

Toplam parametre sayısı: 429504


In [36]:
import json

In [38]:
model_info = {
    "n_estimators": rf_model.n_estimators,
    "classes": rf_model.classes_.tolist(),
    "tree_structure": [
        {
            "node_count": tree.tree_.node_count,
            "children_left": tree.tree_.children_left.tolist(),
            "children_right": tree.tree_.children_right.tolist(),
            "feature": tree.tree_.feature.tolist(),
            "threshold": tree.tree_.threshold.tolist(),
            "value": tree.tree_.value.tolist()
        } for tree in rf_model.estimators_
    ]
}

with open('random_forest_model.json', 'w') as f:
    json.dump(model_info, f)